### Numerical analysis of the lowest energy eigenvalue for an absolute potential well
#### Table of Contents
1. [Introduction](#Introduction)
2. [Interactive plotting!](#iplot)
3. [Error visualization](#evis)
4. [Discussion](#Discussion)

#### Introduction
Consider the following potential well: 
$$ V(x)=a|x|.$$
We can plot this with $a=1$ to get a feel for what it looks like (after imports):

In [1]:
%matplotlib widget
%load_ext autoreload
%autoreload 2
import numpy as np
import matplotlib.pyplot as plt

In [2]:
plt.close('all')
fig, ax = plt.subplots()
x = np.linspace(-1, 1, 100)
line, = plt.plot(x,np.fabs(x), lw=2)
ax.set_xlabel('x')
ax.set_ylabel('V')
plt.title("V(x)=|x|")
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Now, recall the one-dimensional time-independent Schrödinger equation:
$$ -\frac{\hslash^2}{2m}\frac{d^2\psi}{dx^2} +V(x)\psi(x)=E\psi(x).$$
Then, rearranging and plugging in $V(x)=a|x|$, we have:
$$\frac{{d^2 \psi }}{{dx^2 }} + \frac{{2m}}{{\hslash ^2 }}(E - a|x|)\psi  = 0.$$
Now, defining $E = \varepsilon (\hslash ^2 a^2 /m)^{1/3} $ and $ x = z(\hslash ^2 /ma)^{1/3} $ where $\varepsilon$ and $z$ are dimensionless constants, we can substitute and rearrange to find:
$$\begin{equation}
\frac{{d^2 \psi }}{{dz^2 }} + 2(\varepsilon  - |z|)\psi  = 0.
\end{equation}$$

This differential equation is a little hairy to solve analytically, but we can get a feel for a solution using some numerical analysis!
To do this, we convert our differential equation into a recurrence relation, and evaluate at small steps with given initial conditions. We define a step size $\delta$, and determine a $\psi_i$ corresponding to $z=i\delta$ for each $i$ in some reasonable range. Then, using the Taylor series expansions, we have:
$$\begin{align*}
\psi(z+\delta)&=\psi(z)+\left(\frac{d\psi}{dz}\right)_z\delta+\frac{1}{2}\left(\frac{d^2\psi}{dz^2}\right)_z\delta^2+\ldots\\{\rm and}\quad
\psi(z-\delta)&=\psi(z)-\left(\frac{d\psi}{dz}\right)_z\delta+\frac{1}{2}\left(\frac{d^2\psi}{dz^2}\right)_z\delta^2+\ldots.
\end{align*}$$
Substituting the second equation into the first, we obtain:
$$\psi(z+\delta)=-\psi(z-\delta)+2\psi(z)+\left(\frac{d^2\psi}{dz^2}\right)_z\delta^2+\ldots.$$
Then, discarding the higher order terms (which get smaller polynomially) and substituting the second derivative using eq (1), we arrive at our recurrence,
$$\psi_{i+1}=-\psi_{i-1}+2\psi_j-2\delta^2\left(\epsilon-|i\delta|\right)\psi_i.$$

Now, since our lowest energy eigenfunction is even (as required by the parity of $V(x)$) and has no nodes, we know that $d\psi_0/dx=0$ at $z=0$. We can also let $\psi(0)=A$ where $A$ is some normalization constant. Then, we can write a function that determines the value of $\psi_i$:

In [3]:
def waveN(waveNm1, waveNm2, δ, ε, zNm2):
    return -waveNm1 + (2 * waveNm2) - (2 * δ**2 * (ε-np.fabs(zNm2)) * waveNm2)

Now, we also write a function that plots the wavefunction on an interval [0, 4] for some given $\varepsilon$ and number of steps $s$ (where $\delta=\varepsilon/2^s$). 

In addition, we can also calculate the wavefunction's "error" – defining $\psi_{\rm min}$ to be the minimum value achieved by the wavefunction and $\psi_{\rm end}$ to be the value determined for $\psi$ when $z=4$, we have:
$${\rm error}=\begin{cases}
0 & 0 < \psi_{\rm end} < \psi_{\rm min}\\
\psi_{\rm end} - \psi_{\rm min} & \psi_{\rm end} > \psi_{\rm min}\\
|\psi_{\rm end}| & \psi_{\rm end} < 0
\end{cases}.$$

Together, we implement this function as `verifyε` (and define some constants that will be useful later).

In [4]:
def verifyε(steps, ε, wave0=0.7005546694136285, complete=True):
    """Plot the wavefunction numerically using ε and steps.

    Arguments:
    steps       number of steps, log 2, taken from z=0 to z=ε (i.e. where E0=a|x|).
                    i.e. δ = ε * steps^(-2)
    ε           epsilon, the dimensionless parameter related to energy.
    wave0       ψ(0), determining the "amplitude" of the wave.
                    the given value normalizes the solution corresponding to the correct ε,
                    calculated with the help of Mathematica and Wikipedia
    complete    True: returns all values of ψ are desired regardless of correctness.
                False: Truncates ψ when error is detected. Calculates "valid" with remaining number of steps.
                    Plotting with complete=False is not as clear, but saves a lot of time.
    
    Return:
    z       An array of the z-values at which the wavefunction was evaluated. 
                A regular partition of 0 to 4 with step size δ (given above)
    waveQ   An array of the values of the wavefunction corresponding to the given ε,
                calculated numerically with the given δ.
    valid   A positive score denoting how "correct" ε is (given δ), with 0 being "correct"
    """
    # E0 = ε, let hbar^2•a^2/m = 1
    # E0 = a|x| when z = ε

    steps = 2 ** steps
    δ = ε / steps
    z = np.arange(0, 4, δ)
    wave1 = wave0 - δ**2 * ε * wave0
    waveQ = np.zeros((len(z)))
    waveQ[0], waveQ[1] = wave0, wave1
    valid = 0
    for i in range(2, len(z)):
        waveQ[i] = waveN(*waveQ[i-2:i], δ, ε, z[i-2]) 
        valid = 0 if (waveQ[i-2] - waveQ[i] >= 0 and waveQ[i] >= 0) \
                else (1 if not complete else max(waveQ[i]-np.amin(waveQ), -waveQ[i]))
        if not complete and valid != 0:
            valid = len(z)- i
            break

    # calculate the "error" in the wave function 
    # i.e. how negative the tail is, or how much the tail is greater than its minimum value
    return z, waveQ, valid


#### Interactive plotting!
Now, we can play around with the wavefunctions generated by some given steps and ε with an interactive plot (I won't go into the details)! Manipulate the sliders to determine ε and the number of steps, and if you'd like to change the range of the sliders, type in the textboxes. A valid solution will show up as a green curve whereas an invalid solution will display as red.

In [5]:
import matplotlib.colors as colors
import matplotlib.patches as mpatches
from matplotlib.widgets import Slider, Button, TextBox

plt.close("all")
# Plots wavefunction for corresponding ε and δ values (controlled with sliders).
# Red and green correspond to invalid and valid solutions respectively.

εinit = 0.80857 # actual is ±0.00001
εmin = 0.75
εmax = 0.9

stepinit = 6
stepmin = 5
stepmax = 15

# Create the figure and the line that we will manipulate
fig, ax = plt.subplots()
oldx, oldy, valid = verifyε(stepinit, εinit)
color = "green" if valid == 0 else "red"
line, = plt.plot(oldx, oldy, lw=2, color=color)
ax.set_xlabel('z')
ax.set_ylabel('ψ')
plt.title("ψ(x)")
ax.set_ylim([-0.05, 0.8])
plt.xticks(np.arange(0, 4.1, 1))
plt.yticks(np.arange(0, 0.81, 0.2))

# adjust the main plot to make room for the sliders
plt.subplots_adjust(left=0.3, bottom=0.3)


# Make a horizontal slider to control ε (min gets adjusted right before rendering)
axstep = plt.axes([0.3, 0.175, 0.58, 0.03])
step_slider = Slider(
    ax=axstep,
    label="Steps (log2, for δ)",
    valmin=0,
    valmax=stepmax,
    valinit=stepinit,
)

# Make a vertically oriented slider to control δ (min gets adjusted right before rendering)
axε = plt.axes([0.175, 0.3, 0.0225, 0.58])
ε_slider = Slider(
    ax=axε,
    label='ε',
    valmin=0,
    valmax=εmax,
    valinit=εinit,
    orientation="vertical"
)

# The function to be called anytime a slider's value changes
def update(val):
    newx, newy, valid = verifyε(step_slider.val, ε_slider.val)
    line.set_xdata(newx)
    line.set_ydata(newy)
    line.set_color("green" if valid == 0 else "red")
    fig.canvas.draw_idle()

# register the update function with each slider
step_slider.on_changed(update)
ε_slider.on_changed(update)


ax_ε_min = plt.axes([0.05, 0.3, 0.1, 0.04])
εmin_box = TextBox(ax_ε_min, 'ε Min', initial=εmin)

# reposition label
l_εmin = εmin_box.ax.get_children()[1]
l_εmin.set_y(1)
l_εmin.set_verticalalignment('bottom')
l_εmin.set_horizontalalignment('left')

ax_ε_max = plt.axes([0.05, 0.8, 0.1, 0.04])
εmax_box = TextBox(ax_ε_max, 'ε Max', initial=εmax)

# reposition label
l_εmax = εmax_box.ax.get_children()[1]
l_εmax.set_y(1)
l_εmax.set_verticalalignment('bottom')
l_εmax.set_horizontalalignment('left')

ax_step_min = plt.axes([0.3, 0.1, 0.15, 0.04])
smin_box = TextBox(ax_step_min, 'Step Min', initial=stepmin)

ax_step_max = plt.axes([0.75, 0.1, 0.15, 0.04])
smax_box = TextBox(ax_step_max, 'Step Max', initial=stepmax)

def update_slider(text, case):
    step = "s" == case[0]
    try:
        val = float(text)
        slider = step_slider if step else ε_slider
        if "min" in case and 0 < val < slider.valmax:
            slider.valmin = val
            if step:
                slider.ax.set_xlim(val, None)
            else:
                slider.ax.set_ylim(val, None)
            if val > slider.val:
                slider.val=val
                update(val)
        elif "max" in case and val > slider.valmin:
            slider.valmax = val
            if step:
                slider.ax.set_xlim(None, val)
            else:
                slider.ax.set_ylim(None, val)
            if val < slider.val:
                slider.val=val
                update(val)
        fig.canvas.draw_idle()
    except:
        textbox = smin_box if step else εmin_box
        if "min" in case:
            textbox.set_val(slider.valmin)
        else:
            textbox.set_val(slider.valmax)
        pass

smin_box.on_submit(lambda text: update_slider(text, "smin"))
smax_box.on_submit(lambda text: update_slider(text, "smax"))
εmin_box.on_submit(lambda text: update_slider(text, "εmin"))
εmax_box.on_submit(lambda text: update_slider(text, "εmax"))
update_slider(εmin, "εmin")
update_slider(stepmin, "smin")


# Create a `matplotlib.widgets.Button` to reset the sliders to initial values.
resetax = plt.axes([0.8, 0.025, 0.1, 0.04])
button = Button(resetax, 'Reset', hovercolor='0.975')

def reset(event):
    step_slider.reset()
    ε_slider.reset()
    smin_box.set_val(stepmin)
    smax_box.set_val(stepmax)
    εmin_box.set_val(εmin)
    εmax_box.set_val(εmax)
    update_slider(εmin, "εmin")
    update_slider(stepmin, "smin")
    update_slider(εmax, "εmax")
    update_slider(stepmax, "smax")

button.on_clicked(reset)

plt.show()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

#### Error visualization
We can observe two things from the interactive plot:
1. For steps < 13, a lower step count corresponds to an ε neighborhood containing smaller values of ε. Increasing `steps` also shifts up the window for valid values of ε.
2. It seems that around ε = [0.80856, 0.80858] and for steps > 14, our solution starts converging. No matter how much we increase the number of steps, the valid valud for ε doesn't change by much, if at all.

Here, it might be helpful to visualize perhaps a plot of error for different values for "steps" and ε. Let's first focus on the range where it seems to converge: steps = [12, 15] and ε = [0.8085, 0.80865]. We first define a function `getErrors` which runs `verifyε` for a given range of steps and ε's. 

In [6]:
def getErrors(filename, εtol, steptol, **argc):
    try: 
        errors = np.loadtxt(filename)
    except:
        errors = np.zeros((len(εtol), len(steptol)))#I was actually not dumb, wow what a miracle.
        for nε, ε in enumerate(εtol):
            print(nε, "/", len(εtol))
            for nstep, step in enumerate(steptol):
                err = verifyε(step, ε, **argc)[-1]
                errors[nε,nstep] = err
        np.savetxt(filename, errors)
    return errors

Then, we can visualize these errors in a small plot:

In [9]:
# Heatmap for the "error" in the wave function (calculation in line 22 above)
# with respect to various values of ε and δ (close to the actual value of ε)
plt.close("all")

εmin = 0.8085
εmax = 0.80865

stepmin = 12
stepmax = 15

tries = 10
filename = str(tries)+str(tries)+".txt"
εtol = np.linspace(εmin,εmax,tries)
steptol = np.linspace(stepmin,stepmax,tries)
errors = getErrors(filename, εtol, steptol)

extent =[steptol[0],steptol[-1],εtol[0],εtol[-1]]
fig, ax = plt.subplots(figsize=(8,8))
im = ax.imshow(errors, \
    origin='lower',interpolation='none', extent=extent, aspect="auto")
fig.colorbar(im,shrink=0.8)
ax.set_xlabel("Number of steps (log 2)")
ax.set_ylabel("ε value")

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

The correlation is as expected! However, it does still seem to be increasing. To visualize whether the value of ε converges, we can make the same plot but on a larger range of steps and ε and look for an asymptote!

In [10]:
# Heatmap for the "error" in the wave function (calculation in line 22 above)
# with respect to a larger range of values of ε and δ (than above)
plt.close("all")
tries = 100
complete=True
filename = "big"+("" if complete else "n")+str(tries)+str(tries)+".txt"
εtol = np.linspace(0.5,0.9,tries)
steptol = np.linspace(2,15,tries)
errors = getErrors(filename, εtol, steptol)

extent =[steptol[0],steptol[-1],εtol[0],εtol[-1]]
fig, ax = plt.subplots(figsize=(8,8))

im = ax.imshow(errors, \
    origin='lower',interpolation='none', extent=extent, aspect="auto", cmap=plt.get_cmap("plasma"), norm=colors.PowerNorm(0.6))
fig.colorbar(im,shrink=0.8)
ax.set_xlabel("Number of steps (log 2)")
ax.set_ylabel("ε value")

# to show region in more zoomed in plot (2nd if statement)
rect=mpatches.Rectangle((12, εmin),3,εmax-εmin, 
                    fill=False,
                    color="white",
                    linewidth=1)
                    #facecolor="red")
plt.gca().add_patch(rect)

plt.show()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

It looks like there is an asymptote! And the relationship we determined earlier for steps < 13 (that the ε window shifts for greater steps) also seems to hold!

#### Discussion

Though we solved it here numerically, the time-independent Schrödinger equation corresponding to our given potential function has analytical solutions: Airy functions! A [well written overview](https://mazziotti.uchicago.edu/journal/jain_v.pdf) from Varun Jain. It was his analysis of the ground-state 